In [1]:
# importing libraries 
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error

# 1. Python: Download a timeseries of daily deaths per country

In [2]:
#reading the data from the csv file
#global_ds = pd.read_csv(r'https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

url ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
global_ds = pd.read_csv(url, error_bad_lines=False)

In [3]:
#displaying the raw data imported
global_ds

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,1344,1354,1363,1363,1370,1375,1375,1375,1375,1385
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,205,208,213,219,225,228,230,232,234,238
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,1322,1333,1341,1351,1360,1370,1379,1391,1402,1411
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,52,53,53,53,53,53,53,53,53,53
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,80,80,80,86,86,88,88,90,92,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,104,105,106,106,108,110,113,113,119,120
262,NaN,Western Sahara,24.215500,-12.885800,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
263,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,523,528,528,528,528,530,535,537,539,541
264,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,241,246,246,256,260,260,264,264,269,274


In [4]:
#deleting the columns not required for analysis
global_ds=global_ds.drop(columns=['Province/State','Lat','Long'])
#Renaming the column name
global_ds=global_ds.rename(columns={"Country/Region": "country"})

# 2. Python: Convert the table so that each country and each day is a separate row

In [5]:
#using the melt function to convert the columns to rows 
data=global_ds.melt(id_vars=["country"], 
        var_name="Date", 
        value_name="Death_Count")

In [6]:
#Checking the datatypes
data.dtypes

country        object
Date           object
Death_Count     int64
dtype: object

In [7]:
#converting the Datetype of Date column from object to datetime 
data['Date'] = pd.to_datetime(data['Date'])

In [8]:
#Clubing the different regions within the country to get country wise data
data = data.groupby(['country','Date'], as_index=False, sort=False)['Death_Count'].sum()

In [9]:
#displaying the final dataset 
data

,country,Date,Death_Count
0,Afghanistan,2020-01-22,0
1,Albania,2020-01-22,0
2,Algeria,2020-01-22,0
3,Andorra,2020-01-22,0
4,Angola,2020-01-22,0
...,...,...,...
39851,West Bank and Gaza,2020-08-20,120
39852,Western Sahara,2020-08-20,1
39853,Yemen,2020-08-20,541
39854,Zambia,2020-08-20,274


# 3. Python: Provide code to upload the table from step 3 into an SQL table named deaths_total

In [10]:
try:
    sqliteConnection = sqlite3.connect('Covid19.db')
    c = sqliteConnection.cursor()
    c.execute('DROP TABLE IF EXISTS deaths_total')
    c.execute('CREATE TABLE deaths_total(country,Date,Death_Count)')
    data.to_sql('deaths_total', sqliteConnection, if_exists='replace', index = False)
    sqliteConnection.commit()
    c.close()
except sqlite3.Error as error:
    print("Error while executing sqlite script", error)
finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("sqlite connection is closed")

sqlite connection is closed


# 4. Python: From the data in step 2, calculate the daily change in deaths for each country

In [11]:
data['Daily_Deaths'] = ((data['Death_Count']-data.sort_values(by=['Date'], ascending=True)
                       .groupby(['country'])['Death_Count'].shift(1,fill_value=0)))

In [12]:
#displaying the first 10 rows displaying daily deaths in China
data[data['country']=='China'].head(10)

,country,Date,Death_Count,Daily_Deaths
36,China,2020-01-22,17,17
224,China,2020-01-23,18,1
412,China,2020-01-24,26,8
600,China,2020-01-25,42,16
788,China,2020-01-26,56,14
976,China,2020-01-27,82,26
1164,China,2020-01-28,131,49
1352,China,2020-01-29,133,2
1540,China,2020-01-30,171,38
1728,China,2020-01-31,213,42


In [13]:
#displaying the lasr 10 rows displaying daily deaths in Australia
data[data['country']=='Australia'].tail(10)

,country,Date,Death_Count,Daily_Deaths
37984,Australia,2020-08-11,352,21
38172,Australia,2020-08-12,361,9
38360,Australia,2020-08-13,375,14
38548,Australia,2020-08-14,379,4
38736,Australia,2020-08-15,396,17
38924,Australia,2020-08-16,421,25
39112,Australia,2020-08-17,438,17
39300,Australia,2020-08-18,450,12
39488,Australia,2020-08-19,463,13
39676,Australia,2020-08-20,472,9


# 5. Python: Provide code to upload the table from step 4 into an SQL table named deaths_change_python

In [14]:
try:
    sqliteConnection = sqlite3.connect('Covid19.db')
    c = sqliteConnection.cursor()
    c.execute('DROP TABLE IF EXISTS deaths_change_python')
    c.execute('CREATE TABLE deaths_change_python(country,Date,Death_Count,Daily_Deaths)')
    data.to_sql('deaths_change_python', sqliteConnection, if_exists='replace', index = False)
    sqliteConnection.commit()
    c.close()
except sqlite3.Error as error:
    print("Error while executing sqlite script", error)
finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("sqlite connection is closed")

sqlite connection is closed


# 6. SQL: Provide SQL code to calculate the daily change for each country using only the data from deaths_total and save it into an SQL table named deaths_change_sql

In [15]:
try:
    sqliteConnection = sqlite3.connect('Covid19.db')
    c = sqliteConnection.cursor()
    c.execute('DROP TABLE IF EXISTS deaths_change_sql')
    c.execute("CREATE TABLE deaths_change_sql AS SELECT country,Date,Death_Count,Death_Count-lag(Death_Count, 1, 0) OVER(PARTITION BY Country ORDER BY Date) as Daily_Deaths from deaths_total")
    sqliteConnection.commit()
    c.close()
except sqlite3.Error as error:
    print("Error while executing sqlite script", error)
finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("sqlite connection is closed")

sqlite connection is closed
